In [1]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from time import time

Cargar Modelos originales

In [2]:
modeloD2V = Doc2Vec.load("./Modelos/modelo2_FINALD2V.model")
modeloW2V = Word2Vec.load("./Modelos/modelo10FINAL.model")

Obtener Vectores e inferencia

In [ ]:
#GUARDAR MODELO SIN INFERENCIA
modeloD2V.delete_temporary_training_data(keep_doctags_vectors=False, keep_inference=False)
modeloD2V.save('modelofree.model')
modeloD2V.trainables.save('neural.mm')


# Inferir Vectores

In [ ]:
#Vectores y matriz de inferencia
red = modeloD2V.trainables.save('neural.mm')
vectorlockf = red.vectors_lockf
syn1neg = red.syn1neg
del(red)

In [ ]:
modelo.trainables.vectors_lockf = vectorlockf
modelo.trainables.syn1neg = syn1neg
del(vectorlockf)
del(syn1neg)

In [ ]:
modelo.infer_vector(['hola'])

# Función similiar w2v

In [ ]:
keyedvectors = modeloW2V.wv
#keyedvectors.save('vectors.kv')
del(modeloW2V)

##### modelo original

In [ ]:
words = ['gestor', 'conocimiento']
#Modelo W2v
similar_words = keyedvectors.most_similar(words, topn=10)
similar_words

##### Pesos a las palabras y encontrar la media

In [29]:
from numpy import (
    dot, float32 as REAL
)
import numpy as np

from gensim import matutils

mean = [keyedvectors.get_vector('conocimiento')*1.0,keyedvectors.get_vector('gestor')*1.0]
array = np.array(mean)
#mean = matutils.unitvec(array.mean(axis=0)).astype(REAL)
array.mean(axis=0)

NameError: name 'keyedvectors' is not defined

##### Vectores normalizados

In [ ]:
from numpy import linalg as LA
norms=[]
for vector in keyedvectors.vectors:
    norms.append(LA.norm(vector))

##### Producto punto y normalizar 
Ordenar de mayor a menor y su cantidad

In [ ]:
dist = np.dot(keyedvectors.vectors[0:None], mean) / norms[0:None]
best = matutils.argsort(dist, topn=10 + 2, reverse=True) 
best

##### Encontrar la palabra

In [ ]:
keyedvectors.index2word[4933]

# Funcion similar d2v

In [61]:
len(modeloD2V.docvecs.vectors_docs_norm[1])

300

In [85]:
infer = modeloD2V.infer_vector(['conocimiento'])
modeloD2V.docvecs.most_similar([infer], topn=10)

[(215, 0.6413873434066772),
 (416, 0.608231782913208),
 (267, 0.6046205163002014),
 (437, 0.5960988998413086),
 (10, 0.5945175886154175),
 (200, 0.5925189256668091),
 (237, 0.5828874111175537),
 (20, 0.5821707248687744),
 (375, 0.5795838832855225),
 (157, 0.5784246921539307)]

In [88]:
from numpy import (
    dot, float32 as REAL
)
import numpy as np

from gensim import matutils

mean = [infer*1.0]
array = np.array(mean)
mean = matutils.unitvec(array.mean(axis=0)).astype(REAL)
#array.mean(axis=0)

##### Vectores normalizados

In [30]:
from numpy import linalg as LA
norms=[]
for vector in modeloD2V.docvecs.vectors:
    norms.append(LA.norm(vector))

##### Producto punto y normalizar 
Ordenar de mayor a menor y su cantidad

In [89]:
dist = np.dot(modeloD2V.docvecs.vectors_docs_norm[0:None], mean)
best = matutils.argsort(dist, topn=10 + 2, reverse=True) 
best

array([215, 416, 267, 437,  10, 200, 237,  20, 375, 157, 271, 291])

##### Encontrar la palabra

In [ ]:
keyedvectors.index2word[4933]

# Vectores a ElasticSearch

In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch()
es.ping()

True

**creación de indices Palabras**

In [9]:
mapping_palabras="""
{
    "mappings":{
        "properties":{
            "index":{"type": "integer"},
            "palabra":{"type": "keyword"},
            "vector":{
                "type": "dense_vector",
                "dims": 300
            },
            "norm_vec":{
                "type": "dense_vector",
                "dims": 300
            }
        }
    }
}
"""
# es.indices.delete(index='palabras')
es.indices.create(body=mapping_palabras, index='palabras')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'palabras'}

**creación de indices Documentos**

In [67]:
mapping_docs="""
{
    "mappings":{
        "properties":{
            "tag":{"type": "integer"},
            "vector":{
                "type": "dense_vector",
                "dims": 300
            },
            "norm_vec":{
                "type": "dense_vector",
                "dims": 300
            }
        }
    }
}
"""
# es.indices.delete(index='documentos')
es.indices.create(body=mapping_docs, index='documentos')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'documentos'}

In [13]:
print(es.indices.get('palabras'), end='\n\n')
print(es.indices.get('documentos'))

{'palabras': {'aliases': {}, 'mappings': {'properties': {'index': {'type': 'integer'}, 'norm_vec': {'type': 'dense_vector', 'dims': 300}, 'palabra': {'type': 'keyword'}, 'vector': {'type': 'dense_vector', 'dims': 300}}}, 'settings': {'index': {'creation_date': '1608162403100', 'number_of_shards': '1', 'number_of_replicas': '1', 'uuid': 'v2mqowdGSuGXzRYCOihh3A', 'version': {'created': '7060099'}, 'provided_name': 'palabras'}}}}

{'documentos': {'aliases': {}, 'mappings': {'properties': {'norm_vec': {'type': 'dense_vector', 'dims': 300}, 'tag': {'type': 'integer'}, 'vector': {'type': 'dense_vector', 'dims': 300}}}, 'settings': {'index': {'creation_date': '1607123586198', 'number_of_shards': '1', 'number_of_replicas': '1', 'uuid': 'hFkHM_O3TWmms3zkhR36BQ', 'version': {'created': '7060099'}, 'provided_name': 'documentos'}}}}


**Datos para Palabras**

In [82]:
for i, (vector, vector_norm, palabra) in enumerate(zip(modeloW2V.wv.vectors, modeloW2V.wv.vectors_norm,modeloW2V.wv.vocab.keys())):

    data = {
        "index":i,
        "palabra":palabra,
        "vector":vector,
        "norm_vec":vector_norm
    }

    es.index(index="palabras", body=data, id=i)

print(es.count(index='palabras'))

**Datos Documentos D2V**

In [108]:
for i, (vector, vector_norm) in enumerate(zip(modeloD2V.docvecs.vectors_docs, modeloD2V.docvecs.vectors_docs_norm)):

    data = {
        "tag":i,
        "vector":vector,
        "norm_vec":vector_norm
    }

    es.index(index="documentos", body=data, id=i)

print(es.count(index='documentos'))

{'count': 451, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [77]:
modeloW2V.wv.init_sims()

In [18]:
#indice Enumerate
#print(modeloW2V.wv.vectors) Vector
print(modeloW2V.wv.index2word[1]) # palabra
#from numpy import linalg as 
#LA.norm(vector) Norm Vector

'nariño'

In [17]:
array = [modeloW2V.wv.vectors_norm[1]*1.0]
array

[array([-0.0125217 ,  0.01232493, -0.09510542,  0.03245121,  0.0276857 ,
         0.02566618,  0.07061847,  0.02696814,  0.0088935 ,  0.11625333,
         0.06977809,  0.05276983, -0.12684561,  0.03838859,  0.10287628,
        -0.07816274, -0.02390791, -0.03260178, -0.02424236,  0.02812636,
         0.12942778, -0.11180716,  0.00476846,  0.05713879,  0.13059047,
         0.00018749, -0.00914336,  0.01083683,  0.00910252,  0.08247272,
         0.05373651,  0.0007051 ,  0.011437  , -0.01691145, -0.10644438,
        -0.03954473, -0.04432227,  0.08751375,  0.01707965, -0.0289205 ,
        -0.00749234,  0.0687811 ,  0.07332145, -0.01046571, -0.03993998,
         0.01192035, -0.03088628,  0.00045177,  0.01446927, -0.06940972,
        -0.00417179, -0.01960897, -0.06405704,  0.03820181,  0.03848344,
         0.00920843,  0.03239372, -0.02567745, -0.05019053,  0.04907716,
        -0.0071375 ,  0.00595792, -0.03083108,  0.05917067, -0.13350867,
         0.03162795,  0.04597617, -0.07804945,  0.0

In [19]:
from numpy import (
    dot, float32 as REAL
)
import numpy as np
from gensim import matutils

mean = matutils.unitvec(np.array(array).mean(axis=0)).astype(REAL)
limited = modeloW2V.wv.vectors_norm
dist = np.dot(limited, mean)

In [20]:
best = matutils.argsort(dist, topn=10, reverse=True)
best

array([    1,     5,    12, 10108,  7477,  2105,  8797,  5250,  4333,
       10642])